In [1]:
import pandas as pd
from bert import Ner
from tqdm import tqdm

In [3]:
tqdm.pandas()

C:\Users\gabriel.oliveira.luz\AppData\Local\Continuum\anaconda3\envs\projetofinal2\lib\site-packages\tqdm\std.py:725: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
!pip list

Package              Version            
-------------------- -------------------
argon2-cffi          20.1.0             
async-generator      1.10               
attrs                20.3.0             
backcall             0.2.0              
beautifulsoup4       4.9.3              
bleach               3.2.1              
blis                 0.4.1              
boto3                1.13.8             
botocore             1.16.8             
catalogue            1.0.0              
certifi              2020.6.20          
cffi                 1.14.0             
chardet              3.0.4              
click                7.1.2              
colorama             0.4.4              
cycler               0.10.0             
cymem                2.0.3              
decorator            4.4.2              
defusedxml           0.6.0              
docutils             0.15.2             
en-core-web-lg       2.3.0              
en-core-web-sm       2.3.0              
entrypoints     

In [4]:
df = pd.read_csv('data_test_ber/ner_dataset.csv', encoding="ISO-8859-1")  
df.head(5)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [5]:
df['sentence2'] = df.apply(lambda x: 0 if pd.isna(x['Sentence #']) else 1, axis=1)
df.head()

,Sentence #,Word,POS,Tag,sentence2
0,Sentence: 1,Thousands,NNS,O,1
1,NaN,of,IN,O,0
2,NaN,demonstrators,NNS,O,0
3,NaN,have,VBP,O,0
4,NaN,marched,VBN,O,0


In [6]:
df['sentence2'] = df['sentence2'].cumsum()

In [17]:
df.head(50)

,Sentence #,Word,POS,Tag,sentence2
0,Sentence: 1,Thousands,NNS,O,1
1,NaN,of,IN,O,1
2,NaN,demonstrators,NNS,O,1
3,NaN,have,VBP,O,1
4,NaN,marched,VBN,O,1
5,NaN,through,IN,O,1
6,NaN,London,NNP,B-geo,1
7,NaN,to,TO,O,1
8,NaN,protest,VB,O,1
9,NaN,the,DT,O,1


In [7]:
de_para = {
    'B-gpe': 'B-MISC',
    'I-gpe': 'I-MISC',
    'O-gpe': 'O-MISC',
    'B-tim': 'B-MISC',
    'I-tim': 'I-MISC',
    'O-tim': 'O-MISC',
    'B-art': 'B-MISC',
    'I-art': 'I-MISC',
    'O-art': 'O-MISC',
    'B-eve': 'B-MISC',
    'I-eve': 'I-MISC',
    'O-eve': 'O-MISC',
    'B-nat': 'B-MISC',
    'I-nat': 'I-MISC',
    'O-nat': 'O-MISC',
    'B-geo':'B-LOC',
    'I-geo':'I-LOC',
    'O-geo':'O-LOC',
    'B-org ':'B-ORG',
    'I-org ':'I-ORG',
    '0-org ':'0-ORG',
    'B-per ':'B-PER',
    'I-per ':'I-PER',
    'O-per ':'O-PER'
}

In [8]:
df['Tag'] = df['Tag'].replace(de_para)

In [28]:
df.head(50)

,Sentence #,Word,POS,Tag,sentence2
0,Sentence: 1,Thousands,NNS,O,1
1,NaN,of,IN,O,1
2,NaN,demonstrators,NNS,O,1
3,NaN,have,VBP,O,1
4,NaN,marched,VBN,O,1
5,NaN,through,IN,O,1
6,NaN,London,NNP,B-LOC,1
7,NaN,to,TO,O,1
8,NaN,protest,VB,O,1
9,NaN,the,DT,O,1


In [9]:
model = Ner("out_base/")

In [10]:
out = model.predict('I live in Brazil')
print(out)

[{'word': 'I', 'tag': 'O', 'confidence': 0.9991158843040466}, {'word': 'live', 'tag': 'O', 'confidence': 0.9997305274009705}, {'word': 'in', 'tag': 'O', 'confidence': 0.9997486472129822}, {'word': 'Brazil', 'tag': 'B-LOC', 'confidence': 0.9995967745780945}]


In [11]:
df_to_model = df.groupby(['sentence2'])['Word'].apply(' '.join).reset_index()

In [12]:
df_to_model.head()

,sentence2,Word
0,1,Thousands of demonstrators have marched throug...
1,2,Families of soldiers killed in the conflict jo...
2,3,They marched from the Houses of Parliament to ...
3,4,"Police put the number of marchers at 10,000 wh..."
4,5,The protest comes on the eve of the annual con...


In [18]:
df_to_model = df_to_model.iloc[:200]

In [16]:
df_to_model.count

<bound method DataFrame.count of      sentence2                                               Word
0            1  Thousands of demonstrators have marched throug...
1            2  Families of soldiers killed in the conflict jo...
2            3  They marched from the Houses of Parliament to ...
3            4  Police put the number of marchers at 10,000 wh...
4            5  The protest comes on the eve of the annual con...
..         ...                                                ...
995        996  The wall was built starting in 1961 and came t...
996        997  The wall was breached on November 9 , 1989 as ...
997        998  Thousands of Icelanders marked the 90th annive...
998        999  Hundreds of marchers tried to storm central ba...
999       1000  They left after a tense hour-long standoff wit...

[1000 rows x 2 columns]>

In [19]:
df_to_model['result'] = df_to_model.progress_apply(lambda x: model.predict(x['Word']), axis=1)

100%|██████████| 200/200 [01:26<00:00,  2.31it/s]
C:\Users\gabriel.oliveira.luz\AppData\Local\Continuum\anaconda3\envs\projetofinal2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
df_to_model.head(10)

,sentence2,Word,result
0,1,Thousands of demonstrators have marched throug...,"[{'word': 'Thousands', 'tag': 'O', 'confidence..."
1,2,Families of soldiers killed in the conflict jo...,"[{'word': 'Families', 'tag': 'O', 'confidence'..."
2,3,They marched from the Houses of Parliament to ...,"[{'word': 'They', 'tag': 'O', 'confidence': 0...."
3,4,"Police put the number of marchers at 10,000 wh...","[{'word': 'Police', 'tag': 'O', 'confidence': ..."
4,5,The protest comes on the eve of the annual con...,"[{'word': 'The', 'tag': 'O', 'confidence': 0.9..."
5,6,The party is divided over Britain 's participa...,"[{'word': 'The', 'tag': 'O', 'confidence': 0.9..."
6,7,The London march came ahead of anti-war protes...,"[{'word': 'The', 'tag': 'O', 'confidence': 0.9..."
7,8,The International Atomic Energy Agency is to h...,"[{'word': 'The', 'tag': 'O', 'confidence': 0.9..."
8,9,Iran this week restarted parts of the conversi...,"[{'word': 'Iran', 'tag': 'B-LOC', 'confidence'..."
9,10,Iranian officials say they expect to get acces...,"[{'word': 'Iranian', 'tag': 'B-MISC', 'confide..."


In [27]:
df_ent = df[df['Tag'] != 'O']
df_ent.head(50)

,Sentence #,Word,POS,Tag,sentence2
6,NaN,London,NNP,B-LOC,1
12,NaN,Iraq,NNP,B-LOC,1
18,NaN,British,JJ,B-MISC,1
42,NaN,Bush,NNP,B-per,2
65,NaN,Hyde,NNP,B-LOC,3
66,NaN,Park,NNP,I-LOC,3
94,NaN,Britain,NNP,B-LOC,5
97,NaN,Labor,NNP,B-org,5
98,NaN,Party,NNP,I-org,5
102,NaN,English,JJ,B-MISC,5


In [28]:
df_ent['dict'] = df_ent.apply(lambda x: {x['Word']: x['Tag']}, axis=1)

C:\Users\gabriel.oliveira.luz\AppData\Local\Continuum\anaconda3\envs\projetofinal2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
df_ent.head(10)

,Sentence #,Word,POS,Tag,sentence2,dict
6,NaN,London,NNP,B-LOC,1,{'London': 'B-LOC'}
12,NaN,Iraq,NNP,B-LOC,1,{'Iraq': 'B-LOC'}
18,NaN,British,JJ,B-MISC,1,{'British': 'B-MISC'}
42,NaN,Bush,NNP,B-per,2,{'Bush': 'B-per'}
65,NaN,Hyde,NNP,B-LOC,3,{'Hyde': 'B-LOC'}
66,NaN,Park,NNP,I-LOC,3,{'Park': 'I-LOC'}
94,NaN,Britain,NNP,B-LOC,5,{'Britain': 'B-LOC'}
97,NaN,Labor,NNP,B-org,5,{'Labor': 'B-org'}
98,NaN,Party,NNP,I-org,5,{'Party': 'I-org'}
102,NaN,English,JJ,B-MISC,5,{'English': 'B-MISC'}


In [43]:
df_list = df_ent.groupby(['sentence2'])['dict'].apply(lambda x: list(x))#.reset_index()
#df_ent.head(10)

In [44]:
df_final = pd.merge(df_to_model, df_list.loc[:,['sentence2','list']], how='left', on='sentence2') 
df_final.head(10)

IndexingError: Too many indexers